In [ ]:
# 연습문제 :서울시 응답소 사이트의 시장에게 바란다 게시판 크롤링하기
# https://eungdapso.seoul.go.kr/

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os   # 폴더 생성하기 위해 필요함

#Step 2. 사용자에게 크롤링에 필요한 정보를 입력 받습니다.
print("=" *100)
print("연습문제 : 서울시 응답소 게시판 크롤링하기")
print("=" *100)

query_txt = '서울시응답소'

cnt = int(input('1.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10)

f_dir = input("2.결과 파일을 저장할 폴더명만 쓰세요(기본값:c:\\temp\\):")
if f_dir=='' :
    f_dir ='c:\\temp\\'

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.

s_time = time.time( )

path = "c:/temp/chromedriver_90/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://eungdapso.seoul.go.kr/')
driver.maximize_window()

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

#Step 4. 여러 메뉴 중에서 시장에게 바랍니다 게시판 선택
driver.find_element_by_xpath('//*[@id="gnb"]/div[4]/div[2]/div/a').click()

print("=" *80)

# Step 5.게시물의 정보 수집

no2=[]             # 게시글 번호 컬럼
contents=[]        # 게시글 내용 컬럼

no = 1

for x in range(1,page_cnt+1) :
    print("%s 페이지 내용 수집 시작합니다 =======================" %x)
        
    # 1 페이지에 있는 게시글 10건의 상세 내역 가져오기
    for i in range(1,11) :
        
        if no > cnt :
            break
            
        f = open(ff_name, 'a',encoding='UTF-8')
        
        driver.find_element_by_xpath("""//*[@id="content_cont"]/div[2]/div/form/div[4]/div[%s]/ul/li[2]/a"""  %i).click( )
                               
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        #상담내용 추출하기
        content = soup.find('div','table_inner_desc').get_text().replace("\t","").replace("Q. 상담내용","").strip()
        f.write(str(no)+':'+str(content)+'\n')
        f.close( )
        
        print(no,':',content)
        print('')
   
        no2.append(no)                                   # 게시글 번호에 추가
        contents.append(content.replace("\n","")+'\n')   # 내용에 추가
        
        no += 1
        
        time.sleep(1)
        
        driver.back()
        
    time.sleep(2)        # 페이지 변경 전 2초 대기 


    x += 1
    try :
        driver.find_element_by_link_text("%s" %x).click()
    except :
        driver.find_element_by_xpath('//*[@id="content_cont"]/div[2]/div/div[2]/a').click()

    
# Step 6. 수집된 정보를 다양한 형식의 파일로 저장하기 
seoul = pd.DataFrame()
seoul['번호']=no2
seoul['상담내용']=contents

# csv 형태로 저장하기
seoul.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
seoul.to_excel(fx_name,index=False)

#Step 7. 요약 정보 출력하기
e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

print("\n") 
print("=" *100)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *100)

driver.close( )